In [88]:
from keras.applications import VGG16 , InceptionV3
from keras.applications.resnet import ResNet101
import keras
from keras.models import Model
from keras.layers import Dense,Flatten,Dropout
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory=r"/home/pritam/Downloads/Dataset/Train",target_size=(224,224))
valdata = trdata.flow_from_directory(directory=r"/home/pritam/Downloads/Dataset/Test", target_size=(224,224))
testdata = trdata.flow_from_directory(directory=r"/home/pritam/Downloads/Dataset/TestNew", target_size=(224,224))
    



model = ResNet101(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze four convolution blocks
for layer in model.layers:
    layer.trainable = False
# Make sure you have frozen the correct layers
# for i, layer in enumerate(vgg_model.layers):
#     print(i, layer.name, layer.trainable)
    

    
x = model.output
x = Flatten()(x) # Flatten dimensions to for use in FC layers
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x) # Dropout layer to reduce overfitting

x = Dense(256, activation='relu')(x)
x = Dense(3, activation='softmax')(x) # Softmax for multiclass

transfer_model = Model(inputs= model.input, outputs=x)

transfer_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001 , momentum=0.9), metrics=["accuracy"])

transfer_model.summary()

from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("inception_v3.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=40, verbose=1, mode='auto')
hist = transfer_model.fit_generator(generator= traindata, epochs= 10, validation_data= testdata,  callbacks=[checkpoint,early])
transfer_model.save_weights("resnet2.h5")


import matplotlib.pyplot as plt
print(history1.history.keys())
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('resnet_loss.png')

plt.clf()
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.savefig('resnet_acc.png')

Found 9384 images belonging to 3 classes.
Found 3128 images belonging to 3 classes.
Found 3615 images belonging to 3 classes.
Model: "model_39"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_40 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_40[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                               

/tmp/ipykernel_10416/798805893.py:46: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = transfer_model.fit_generator(generator= traindata, epochs= 10, validation_data= testdata,  callbacks=[checkpoint,early])


Epoch 1/10
 19/294 [>.............................] - ETA: 1:02 - loss: 1.0227 - accuracy: 0.6678

KeyboardInterrupt: 

In [60]:
transfer_model.save("mobile.h5")